Going to focus on every bike into and out of "Waterloo Station 3, Waterloo"

In [3]:
import boto3
import pandas as pd
import os
from tqdm import tqdm

In [4]:
# Get list of csv names
with open("text_files/files.txt", "r") as f:
    CSVs = f.read()

CSVs = CSVs.replace("\n    ", " ").replace("%20", "_")[2:-2].split("', '")

In [18]:
# # Specify your S3 bucket and CSV file paths
# bucket_name = 'tfl-cycle-data'
# csv_dir = 'JourneyInfo/'

# # Using above CSV list when not testing
# # CSVs = ['387JourneyDataExtract01Jan2024-14Jan2024.csv',
# #         '388JourneyDataExtract15Jan2024-31Jan2024.csv',
# #         '389JourneyDataExtract01Feb2024-14Feb2024.csv',
# #         '390JourneyDataExtract15Feb2024-29Feb2024.csv',
# #         '391JourneyDataExtract01Mar2024-14Mar2024.csv',
# #         '392JourneyDataExtract15Mar2024-31Mar2024.csv',
# #         '393JourneyDataExtract01Apr2024-14Apr2024.csv',
# #         '394JourneyDataExtract15Apr2024-30Apr2024.csv']

# cols = []
# # Initialize a session using Amazon S3
# s3 = boto3.client('s3')

# progress_bar = tqdm(CSVs, desc="Processing")

# for idx, csv in enumerate(progress_bar):
#     file_path = f"temp/{csv}"
#     # Download the CSV file from S3
#     s3.download_file(bucket_name, f"{csv_dir}{csv}", file_path)
#     df = pd.read_csv(file_path)
    
#     # Rename columns and keep only 'Rental ID', 'Start Date','Start Station Name',  'End Date', 'End Station Name'
#     Renaming_dict = {'Number': 'Rental ID',
#                      'rental ID': 'Rental ID',
#                      'Rental Id': 'Rental ID',
#                      'Start date': 'Start Date',
#                      'StartStation Name': 'Start Station Name',
#                      'Start station': 'Start Station Name',
#                      'End date': 'End Date',
#                      'EndStation Name': 'End Station Name',
#                      'End station': 'End Station Name'
#                         }
#     df.rename(columns=Renaming_dict, inplace=True)
#     # print(df.columns)
    
#     cols_to_keep = ['Rental ID', 'Start Date', 'Start Station Name',  'End Date', 'End Station Name']
#     df = df[cols_to_keep]
    
#     df['Start Date'] = pd.to_datetime(df['Start Date'], format='%Y-%m-%d %H:%M', errors='coerce')
#     df['End Date'] = pd.to_datetime(df['End Date'], format='%Y-%m-%d %H:%M', errors='coerce')
#     df['Start Station Name'] = df['Start Station Name'].astype(str)
#     df['End Station Name'] = df['End Station Name'].astype(str)
    
        
#     c = " | ".join(list(df.columns))
#     if c not in cols:
#         cols.append(c)
#     os.remove(file_path)
#     # progress_bar.set_description(f"Processing Index: {idx}/{len(CSVs)}, merged_df is {merged_df.memory_usage(deep=True).sum()/(1024*1024)}MB large")
#     progress_bar.set_description(f"Processing Index: {idx}/{len(CSVs)}. Unique Cols: {len(cols)}")
    
# # print(f"merged_df is {merged_df.memory_usage(deep=True).sum()/(1024*1024)}MB large")    

# # merged_df.describe()
# # print(cols)

Processing Index: 289/424. Unique Cols: 1:  68%|██████▊   | 290/424 [07:29<04:09,  1.86s/it]/tmp/ipykernel_26234/3223177310.py:25: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Processing Index: 311/424. Unique Cols: 1:  74%|███████▎  | 312/424 [08:01<02:49,  1.51s/it]/tmp/ipykernel_26234/3223177310.py:25: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Processing Index: 312/424. Unique Cols: 1:  74%|███████▍  | 313/424 [08:03<03:05,  1.67s/it]/tmp/ipykernel_26234/3223177310.py:25: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Processing Index: 423/424. Unique Cols: 1: 100%|██████████| 424/424 [11:47<00:00,  1.67s/it]


In [19]:
# for c in cols:
#     print(c)
    
# # I want cols:
# # 'Rental ID', 'Start Date','Start Station Name',  'End Date', 'End Station Name'

# Renaming_dict = {'Number': 'Rental ID',
#                  'Start date': 'Start Date',
#                  'StartStation Name': 'Start Station Name',
#                  'Start station': 'Start Station Name',
#                  'End date': 'End Date',
#                  'EndStation Name': 'End Station Name',
#                  'End station': 'End Station Name'
#                 }


Rental ID | Start Date | Start Station Name | End Date | End Station Name


In [ ]:
# explore 289, 311, 312 - mixed data types in cols (2,5) start / end station num?

In [6]:
# Specify your S3 bucket and CSV file paths
bucket_name = 'tfl-cycle-data'
csv_dir = 'JourneyInfo/'

merged_df = pd.DataFrame()
cols = []
# Initialize a session using Amazon S3
# s3 = boto3.client('s3')


progress_bar = tqdm(CSVs, desc="Processing")

for idx, csv in enumerate(progress_bar):
    file_path = f"temp/{csv}"
    # Download the CSV file from S3
    # s3.download_file(bucket_name, f"{csv_dir}{csv}", file_path)
    file_path = f"data/Journey_Data/{csv}"
    df = pd.read_csv(file_path, skip_blank_lines=True)
    
    # Rename columns and keep only 'Rental ID', 'Start Date','Start Station Name',  'End Date', 'End Station Name'
    Renaming_dict = {'Number': 'Rental ID',
                     'rental ID': 'Rental ID',
                     'Rental Id': 'Rental ID',
                     'Start date': 'Start Date',
                     'StartStation Name': 'Start Station Name',
                     'Start station': 'Start Station Name',
                     'End date': 'End Date',
                     'EndStation Name': 'End Station Name',
                     'End station': 'End Station Name'
                        }
    df.rename(columns=Renaming_dict, inplace=True)
    # print(df.columns)
    
    cols_to_keep = ['Rental ID', 'Start Date', 'Start Station Name',  'End Date', 'End Station Name']
    df = df[cols_to_keep]
    
    dataFormats = ['%d/%m/%Y %H:%M,%f','%Y-%m-%d %H:%M']
    
    df['Start Date'] = pd.to_datetime(df['Start Date'], errors='coerce')
    df['End Date'] = pd.to_datetime(df['End Date'], errors='coerce')
    df['Start Station Name'] = df['Start Station Name'].astype(str)
    df['End Station Name'] = df['End Station Name'].astype(str)

    # df = df[(df['Start Station Name'] == "Waterloo Station 3, Waterloo") | (df['End Station Name'] == "Waterloo Station 3, Waterloo")]
    merged_df = pd.concat([merged_df, df], ignore_index=True)
    os.remove(file_path)
    progress_bar.set_description(f"Processing Index: {idx+1}/{len(CSVs)}, merged_df is {merged_df.memory_usage(deep=True).sum()/(1024*1024)}MB large")

# merged_df.describe()
# print(cols)

Processing Index: 1/424, merged_df is 52.090529441833496MB large:   0%|          | 1/424 [00:00<04:09,  1.69it/s]/tmp/ipykernel_19769/1755434113.py:39: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Start Date'] = pd.to_datetime(df['Start Date'], errors='coerce')
/tmp/ipykernel_19769/1755434113.py:40: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['End Date'] = pd.to_datetime(df['End Date'], errors='coerce')
Processing Index: 2/424, merged_df is 105.70624446868896MB large:   0%|          | 2/424 [00:01<04:49,  1.46it/s]/tmp/ipykernel_19769/1755434113.py:39: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Start Date']

KeyboardInterrupt: 

In [64]:
merged_df.head()

,Rental ID,Start Date,Start Station Name,End Date,End Station Name
0,50755276,2016-10-01 08:41:00,"William IV Street, Strand",2016-10-01 08:47:00,"Waterloo Station 3, Waterloo"
1,50755420,2016-10-01 09:05:00,"Bermondsey Street, Bermondsey",2016-10-01 09:15:00,"Waterloo Station 3, Waterloo"
2,50755598,2016-10-01 09:28:00,"Lollard Street, Vauxhall",2016-10-01 09:36:00,"Waterloo Station 3, Waterloo"
3,50755603,2016-10-01 09:28:00,"Lollard Street, Vauxhall",2016-10-01 09:36:00,"Waterloo Station 3, Waterloo"
4,50756101,2016-10-01 10:12:00,"Waterloo Station 3, Waterloo",2016-10-01 10:27:00,"Braham Street, Aldgate"


In [65]:
merged_df.dtypes

Rental ID                      int64
Start Date            datetime64[ns]
Start Station Name            object
End Date              datetime64[ns]
End Station Name              object
dtype: object

In [68]:
import pickle

file_path = 'temp/waterloo_file.pkl'
s3_file_key = 'Docking_station_data/Waterloo-Jan2016-Apr2024.pkl'

with open(file_path, 'wb') as f:
    pickle.dump(merged_df, f)

s3 = boto3.client('s3')
s3_bucket_name = 'tfl-cycle-data'
s3.upload_file(file_path, s3_bucket_name, s3_file_key)


In [58]:
file_path = 'temp/waterloo_file.pkl'
with open(file_path, 'rb') as f:
    waterloo_df = pickle.load(f)

c = waterloo_df[waterloo_df['Start Date'].isna()]

waterloo_df_notNAT = waterloo_df[~(waterloo_df['Start Date'].isna())]

waterloo_df_notNAT.head()

,Rental ID,Start Date,Start Station Name,End Date,End Station Name
714793,125229627,2022-09-18 22:59:00,"Waterloo Station 3, Waterloo",2022-09-18 23:03:00,"LSBU (Borough Road), Elephant & Castle"
714794,125229481,2022-09-18 22:43:00,"Waterloo Station 3, Waterloo",2022-09-18 22:54:00,"West Smithfield Rotunda, Farringdon"
714795,125229371,2022-09-18 22:34:00,"Waterloo Station 3, Waterloo",2022-09-18 22:47:00,"Northington Street , Holborn"
714796,125229257,2022-09-18 22:22:00,"Waterloo Station 3, Waterloo",2022-09-18 22:47:00,"Wellington Row, Bethnal Green"
714797,125228777,2022-09-18 21:43:00,"Waterloo Station 3, Waterloo",2022-09-18 22:06:00,"Waterloo Station 3, Waterloo"


In [6]:
# to be deleted

# Path to the directory containing CSV files
directory_path = 'temp/'

# List to hold dataframes
dataframes = []

# Read all CSV files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)
        dataframes.append(df)

# Merge all dataframes into a single dataframe
merged_dataframe = pd.concat(dataframes, ignore_index=True)

In [7]:
merged_dataframe.head()

,Number,Start date,Start station number,Start station,End date,End station number,End station,Bike number,Bike model,Total duration,Total duration (ms)
0,138583046,2024-04-14 23:58,1190,"Kennington Lane Rail Bridge, Vauxhall",2024-04-15 00:01,1073,"Kennington Road Post Office, Oval",50789,CLASSIC,3m 27s,207988
1,138583048,2024-04-14 23:58,993,"Drummond Street , Euston",2024-04-15 00:16,1227,"Devonshire Terrace, Bayswater",55839,CLASSIC,17m 52s,1072179
2,138583049,2024-04-14 23:58,1090,"Warren Street Station, Euston",2024-04-15 00:05,978,"Russell Square Station, Bloomsbury",60337,PBSC_EBIKE,6m 32s,392736
3,138583050,2024-04-14 23:58,200235,"Manbre Road, Hammersmith",2024-04-15 00:12,1016,"Hereford Road, Bayswater",60193,PBSC_EBIKE,13m 37s,817269
4,138583042,2024-04-14 23:57,200175,"Wandsworth Rd, Isley Court, Wandsworth Road",2024-04-15 00:05,300006,"Clapham South, Clapham South",60358,PBSC_EBIKE,8m 17s,497745


In [9]:
merged_dataframe.tail()

,Number,Start date,Start station number,Start station,End date,End station number,End station,Bike number,Bike model,Total duration,Total duration (ms)
2454642,136450337,2024-01-01 00:03,1034,"Bruton Street, Mayfair",2024-01-01 01:29,3452,"Panton Street, West End",60538,PBSC_EBIKE,1h 26m 2s,5162949
2454643,136450332,2024-01-01 00:01,300235,"York Way, Kings Cross",2024-01-01 00:31,200212,"Blackfriars Station, St. Paul's",54136,CLASSIC,29m 54s,1794054
2454644,136450333,2024-01-01 00:01,300235,"York Way, Kings Cross",2024-01-01 00:31,200212,"Blackfriars Station, St. Paul's",57461,CLASSIC,29m 43s,1783017
2454645,136450334,2024-01-01 00:01,300235,"York Way, Kings Cross",2024-01-01 00:30,200212,"Blackfriars Station, St. Paul's",21426,CLASSIC,29m 23s,1763701
2454646,136450335,2024-01-01 00:02,200132,"Gaywood Street, Elephant & Castle",2024-01-01 00:04,1076,"Walnut Tree Walk, Vauxhall",56710,CLASSIC,2m 56s,176167


In [29]:
# Useful code but not needed since I am filtering out to just Waterloo for now.

# # get unique station number name lookup
# docks = merged_dataframe[['Start station number', 'Start station']].drop_duplicates()
# docks.columns = ['stationID', 'StationName']
# docks = docks.sort_values('stationID').reset_index(drop=True)
# print(docks)
# dock_locs = pd.read_csv("docking_station_locations.txt")
# print(dock_locs)

# docks.merge(dock_locs, how='left', on='StationName')
# print(docks)

     stationID                        StationName
0          959            Milroy Walk, South Bank
1          960          Hop Exchange, The Borough
2          961          Union Street, The Borough
3          962        Stamford Street, South Bank
4          963             Bankside Mix, Bankside
..         ...                                ...
798     300248        Sopwith Way, Battersea Park
799     300249      Westminster Pier, Westminster
800     300250             Temple Gardens, Temple
801     300252  Canada Water Station, Rotherhithe
802     300253     Bermondsey Station, Bermondsey

[803 rows x 2 columns]
     Station.Id                           StationName  longitude   latitude  \
0             1             River Street, Clerkenwell  -0.109971  51.529200   
1             2        Phillimore Gardens, Kensington  -0.197574  51.499600   
2             3  Christopher Street, Liverpool Street  -0.084606  51.521300   
3             4       St. Chad's Street, King's Cross  -0.12

In [15]:
# Dataframes representing inflows and outflows

docks_2024_out = merged_dataframe[['Start Date', 'Start Station Name']]
docks_2024_in = merged_dataframe[['End Date', 'End Station Name']]

In [20]:
# df = docks_2024_in.groupby('stationName').agg('count')
# print(df.sort_values('stationID'))

                                               date_time  stationID
stationName                                                        
Eaton Square, Belgravia                               10         10
Mechanical Workshop Penton                            16         16
Mechanical Workshop Clapham                          131        131
Here East South, Queen Elizabeth Olympic Park        283        283
Colombo Street, Southwark                            302        302
...                                                  ...        ...
Wormwood Street, Liverpool Street                  11617      11617
Hop Exchange, The Borough                          12183      12183
Brushfield Street, Liverpool Street                12376      12376
Hyde Park Corner, Hyde Park                        13468      13468
Waterloo Station 3, Waterloo                       17914      17914

[805 rows x 2 columns]
